In [28]:
!pip install langchain

In [29]:
!pip install langchain langchain_community


In [30]:
!!pip install langchain-groq


['Requirement already satisfied: langchain-groq in /usr/local/lib/python3.11/dist-packages (0.3.2)',
 'Requirement already satisfied: langchain-core<1.0.0,>=0.3.49 in /usr/local/lib/python3.11/dist-packages (from langchain-groq) (0.3.55)',
 'Requirement already satisfied: groq<1,>=0.4.1 in /usr/local/lib/python3.11/dist-packages (from langchain-groq) (0.23.1)',
 'Requirement already satisfied: anyio<5,>=3.5.0 in /usr/local/lib/python3.11/dist-packages (from groq<1,>=0.4.1->langchain-groq) (4.9.0)',
 'Requirement already satisfied: distro<2,>=1.7.0 in /usr/local/lib/python3.11/dist-packages (from groq<1,>=0.4.1->langchain-groq) (1.9.0)',
 'Requirement already satisfied: httpx<1,>=0.23.0 in /usr/local/lib/python3.11/dist-packages (from groq<1,>=0.4.1->langchain-groq) (0.28.1)',
 'Requirement already satisfied: pydantic<3,>=1.9.0 in /usr/local/lib/python3.11/dist-packages (from groq<1,>=0.4.1->langchain-groq) (2.11.3)',
 'Requirement already satisfied: sniffio in /usr/local/lib/python3.11

In [31]:
# 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

import os

os.environ["GROQ_API_KEY"] = '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'


In [32]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

intent_prompt = PromptTemplate.from_template(
    "Is the following query medically related? Reply with yes or no only:\n\n{query}"
)
chain = intent_prompt | llm | StrOutputParser()

def is_medical_query(query: str) -> bool:
    result = chain.invoke({"query": query})
    return result.strip().lower() == "yes"


In [33]:
from langchain.agents import initialize_agent, Tool
from langchain.agents.agent_types import AgentType
from langchain.tools import WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    model_name="llama3-70b-8192"
)

from langchain_community.agent_toolkits.sql.base import SQLDatabaseToolkit
import asyncio
from langchain.sql_database import SQLDatabase
from langchain.tools.wolfram_alpha import WolframAlphaQueryRun
from langchain.utilities.wolfram_alpha import WolframAlphaAPIWrapper
from langchain.tools import BaseTool
from typing import Optional
import os
import nest_asyncio

In [34]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

intent_prompt = PromptTemplate.from_template(
    "Is the following query medically related? Reply with yes or no only:\n\n{query}"
)
chain = intent_prompt | llm | StrOutputParser()

def is_medical_query(query: str) -> bool:
    result = chain.invoke({"query": query})
    return result.strip().lower() == "yes"

In [35]:
nest_asyncio.apply()

In [36]:
from langchain.agents import initialize_agent, Tool
from langchain.agents.agent_types import AgentType
from langchain.tools import WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
from langchain_community.agent_toolkits.sql.base import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.tools.wolfram_alpha import WolframAlphaQueryRun
from langchain.utilities.wolfram_alpha import WolframAlphaAPIWrapper
from langchain.tools import BaseTool
from langchain_groq import ChatGroq

from typing import Optional
import asyncio
import os
import nest_asyncio

nest_asyncio.apply()


In [37]:
# SET YOUR API KEYS
os.environ["GROQ_API_KEY"] = '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'
os.environ["WOLFRAM_ALPHA_APPID"] = "HGY66R-VYLKYYVH5R"       #GY66R-VYLKYYVH5R
os.environ["SERPER_API_KEY"] = "0462b1d01faf3b41941417c9e3169d98a52456b4"     #0462b1d01faf3b41941417c9e3169d98a52456b4

In [38]:
# 🧠 Groq LLM using LLaMA 3
llm = ChatGroq(
    temperature=0,
    model_name="llama3-70b-8192"
)


In [39]:
# 1. Google Serper Tool (custom wrapper)
class GoogleSerperTool(BaseTool):
    name: str = "Google Serper"
    description: str = "Use this to search for clinical studies and news using Serper"

    def _run(self, query: str) -> str:
        import requests
        headers = {
            "X-API-KEY": os.getenv("SERPER_API_KEY"),
            "Content-Type": "application/json",
        }
        data = {"q": query}
        response = requests.post(
            "https://google.serper.dev/search", headers=headers, json=data
        )
        return response.json().get("organic", [])[0].get("snippet", "No result")

    def _arun(self, query: str) -> str:
        raise NotImplementedError("Async not supported")

In [40]:
!pip install wikipedia

In [41]:
# 2. Wikipedia Tool
wikipedia_tool = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())



In [42]:
# 3. SQL Database Tool
db = SQLDatabase.from_uri("sqlite:///hospital_rules.db")
sql_toolkit = SQLDatabaseToolkit(db=db, llm=llm)
sql_tool = sql_toolkit.get_tools()[0]  # usually the query tool

In [43]:
# 4. Jina Search (placeholder)
class JinaSearchTool(BaseTool):
    name: str = "Jina Search"
    description: str = "Looks for recent FDA warnings via semantic search"

    def _run(self, query: str) -> str:
        return "Recent FDA warning found regarding ibuprofen interaction with ACE inhibitors."

    def _arun(self, query: str) -> str:
        raise NotImplementedError("Async not supported")

In [44]:
!pip install wolframalpha


In [45]:
# 5. Wolfram Alpha Tool
wolfram_tool = WolframAlphaQueryRun(api_wrapper=WolframAlphaAPIWrapper())

In [46]:
# Combine tools
tools = [
    Tool.from_function(
        func=GoogleSerperTool()._run,
        name="Google Serper",
        description="Check for clinical studies on kidney risks",
    ),
    Tool.from_function(
        func=wikipedia_tool.run,
        name="Wikipedia",
        description="Look up drug mechanisms",
    ),
    Tool.from_function(
        func=sql_tool.run,
        name="SQLDatabase",
        description="Check hospital prescribing rules",
    ),
    Tool.from_function(
        func=JinaSearchTool()._run,
        name="Jina Search",
        description="Look for recent FDA warnings",
    ),
    Tool.from_function(
        func=wolfram_tool.run,
        name="Wolfram Alpha",
        description="Calculate kidney function risk score",
    ),
]

In [47]:
# Initialize Agent
agent_executor = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

In [48]:
# Async query runner
async def main():
    query = "I have pain in my stomach. what to do. what medicine should i take ?"
    response = await agent_executor.ainvoke(query)
    print("\n🧠 ChatBot Response:\n", response)

In [50]:
user_input = input("User: ")

if not is_medical_query(user_input):
    print("Invalid query: This chatbot only handles medical questions.")
else:
    response = agent_executor.invoke({"input": user_input})
    print("Bot:", response)


User: Can I eat spicy food when i have sores in somach


> Entering new AgentExecutor chain...
Thought: This question seems to be related to stomach health and food choices. I need to determine if spicy food is safe to consume when having stomach sores.

Action: Google Serper
Action Input: "stomach sores and spicy food"
Observation: For decades, spicy foods were blamed for painful stomach ulcers. People doubled over in pain were advised to cut back on fiery foods.
Thought:Thought: The observation suggests that spicy food was previously thought to exacerbate stomach ulcers, but I need to know if this is still the case and if it applies to stomach sores in general.

Action: Wikipedia
Action Input: "stomach sores"
Observation: Page: Abdominal pain
Summary: Abdominal pain, also known as a stomach ache, is a symptom associated with both non-serious and serious medical issues. Since the abdomen contains most of the body's vital organs, it can be an indicator of a wide variety of diseases. Gi

In [ ]:
# Async query runner
async def main():
    query = "I have pain in my stomach. what to do. what medicine should i take ?"
    response = await agent_executor.ainvoke(query)
    print("\n🧠 ChatBot Response:\n", response)